In [1]:
using PanTaGruEl

[ Info: Precompiling PanTaGruEl [62397bf7-b360-4750-b643-3b7e4fbfe5ff]


Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-24
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-24


In [3]:
country = ["AL", "AT", "BA", "BE", "BG", "CH", "CZ", "DE", "ES",
    "FR", "GR", "HR", "HU", "IL", "IT", "LU", "ME", "MK",
    "NL", "PL", "PT", "RO", "RS", "SA", "SI", "SK", "DK"];

In [4]:
source_folder = "../sources"
output_folder = "../tmp/Europe"
file_name = "entsoe"

"entsoe"

In [5]:
create_entsoe_list(source_folder, output_folder, file_name, country)

Handling file 1
Handling file 2
Handling file 3
Handling file 4
Handling file 5
Handling file 6
Handling file 7
Handling file 8
Handling file 9
Handling file 10
Handling file 11
Handling file 12
Handling file 13
Handling file 14
Handling file 15
Handling file 16
Handling file 17
Handling file 18
Handling file 19
Handling file 20
Handling file 21
Handling file 22
Handling file 23
Handling file 24
Handling file 25
Handling file 26
Handling file 27
Handling file 28
Handling file 29
Handling file 30
Handling file 31
Handling file 32
Handling file 33
Handling file 34
Handling file 35
Handling file 36
Handling file 37
Handling file 38
Handling file 39
Handling file 40
Handling file 41
Handling file 42
Handling file 43
Handling file 44
Handling file 45
Handling file 46
Handling file 47
Handling file 48
Handling file 49
Handling file 50
Handling file 51
Handling file 52
Handling file 53
Handling file 54
Handling file 55
Handling file 56
Handling file 57
Handling file 58
Handling file 59
Handli

"../tmp/Europe/entsoe.csv"